<h1> Reddit Scraper Utility

In [1]:
import praw
import os
import sys
from PIL import Image
import urllib.request
# import posixpath
# import urllib.parse
# import requests

In [2]:
API_CLIENT_ID = os.environ.get("PUBLIC_KEY_CLIENT_ID")
API_CLIENT_SECRET = os.environ.get("PUBLIC_KEY_CLIENT_SECRET")
API_USER_AGENT = os.environ.get("PUBLIC_KEY_USER_AGENT")

In [3]:
reddit = praw.Reddit(client_id=API_CLIENT_ID, client_secret=API_CLIENT_SECRET, user_agent=API_USER_AGENT)

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the `Reddit` class constructor, or as an environment variable.

In [4]:
number_to_scrape = 5
hot_posts = reddit.subreddit('Sneakers').hot(limit=number_to_scrape)
for post in hot_posts:
    print(post.title)

NameError: name 'reddit' is not defined

In [5]:
import pandas as pd
posts = []
sneakers_subreddit = reddit.subreddit('Sneakers')
for post in sneakers_subreddit.hot(limit=number_to_scrape):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts.head(50)

NameError: name 'reddit' is not defined

In [6]:
# scope of the project: why is this important? could we use this to begin to see which brands and trends are on the come up?
# drop all non-jpg rows?
# how to link data to upvotes
# convert date column to a readable datetime format
# Brands to decipher: Nike, Adidas, New_Balance
# See if I can scrape from a certain time frame
# ^ check to see if Reddit is OK with me querying this much data

In [7]:
# iterate through all queried posts and download the corresponding image as a jpg file

for i in range(1, len(posts['url'])):
    try:
        urllib.request.urlretrieve(posts['url'][i], "raw_data/0000"+str(i)+".jpg")
    except:
        pass

TypeError: list indices must be integers or slices, not str

In [8]:
# resize all of the downloaded images for better scaling effects for the downstream convolutional neural network
# https://stackoverflow.com/questions/21517879/python-pil-resize-all-images-in-a-folder
# https://kishstats.com/python/2018/04/27/bulk-image-resizing-python.html
# https://stackoverflow.com/questions/22282760/filenotfounderror-errno-2-no-such-file-or-directory

path = "/Users/Oliver/GDrive/Data_Scientist_Career/Projects/Hype/raw_data"

for image in os.listdir(path):
    try:
        current_img = Image.open(path+"/"+image)
        f, e = os.path.splitext(image)
        resize_img = current_img.resize((200,200), Image.ANTIALIAS)
        resize_img.save('resized_data/' + f +'-resized.jpg', 'JPEG', quality = 90)
    except:
        pass